In [ ]:
import numpy as np
import xgboost as xgb
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection

from bayeso import bo
from bayeso.wrappers import wrappers_bo
from bayeso.utils import utils_plotting
from bayeso.utils import utils_bo

In [ ]:
digits = sklearn.datasets.load_digits()
data_digits = digits.images
data_digits = np.reshape(data_digits,
    (data_digits.shape[0], data_digits.shape[1] * data_digits.shape[2]))
labels_digits = digits.target

data_train, data_test, labels_train, labels_test = sklearn.model_selection.train_test_split(
    data_digits, labels_digits, test_size=0.3, stratify=labels_digits)

In [ ]:
def fun_target(bx):
    model_xgb = xgb.XGBClassifier(
        max_depth=int(bx[0]),
        n_estimators=int(bx[1]),
        use_label_encoder=False
    )
    model_xgb.fit(data_train, labels_train, eval_metric='mlogloss')
    preds_test = model_xgb.predict(data_test)
    return 1.0 - sklearn.metrics.accuracy_score(labels_test, preds_test)

In [ ]:
str_fun = 'xgboost'

# (max_depth, n_estimators)
bounds = np.array([[1, 10], [100, 500]])
num_bo = 5
num_iter = 25
num_init = 1

In [ ]:
model_bo = bo.BO(bounds, debug=False)

list_X = []
list_Y = []
list_time = []

for ind_bo in range(0, num_bo):
    print('BO Round:', ind_bo + 1)
    X_final, Y_final, time_final, _, _ = wrappers_bo.run_single_round(
        model_bo, fun_target, num_init, num_iter,
        seed=42 * ind_bo)
    list_X.append(X_final)
    list_Y.append(Y_final)
    list_time.append(time_final)

arr_X = np.array(list_X)
arr_Y = np.array(list_Y)
arr_time = np.array(list_time)

arr_Y = np.expand_dims(np.squeeze(arr_Y), axis=0)
arr_time = np.expand_dims(arr_time, axis=0)

In [ ]:
for ind_bo in range(0, num_bo):
    bx_best, y_best = utils_bo.get_best_acquisition_by_history(arr_X[ind_bo],
                                                               arr_Y[0, ind_bo][..., np.newaxis])
    
    print('BO Round', ind_bo + 1)
    print(bx_best, y_best)

In [ ]:
utils_plotting.plot_minimum_vs_iter(arr_Y, [str_fun], num_init, True,
    str_x_axis='Iteration',
    str_y_axis='1 - Accuracy')
utils_plotting.plot_minimum_vs_time(arr_time, arr_Y, [str_fun], num_init, True,
    str_x_axis='Time (sec.)',
    str_y_axis='1 - Accuracy')